In [430]:
import math
import random

sc = SparkContext.getOrCreate()

TRAIN_TEST = 0.8

In [431]:
global n_columns
n_columns=56 #total number of columns - the one deleted - the target = 56
col_sums=[]
averages=[]
sigmas=[]


def initializeAccumulators():
    i=0
    while(i<n_columns):
        averages.append(sc.accumulator(0))
        col_sums.append(sc.accumulator(0))
        sigmas.append(sc.accumulator(0)) 
        i+=1

def addToAccumulators(row):
    if(len(row)-1!=len(col_sums)):
        raise Exception("Number of columns in the row doesn't mach the number of accumulators initiated. Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
    i=0
    while(i<n_columns):
        print("Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
        col_sums[i].add(row[i])
        i+=1
        
def preprocessing(row):
    #splitting
    split=row.split(" ")
    #del 56 col
    del split[56]
    split=[float(col) for col in split]
    addToAccumulators(split)
    #assign random key to the datapoint
    key=random.getrandbits(64)
    #assign train/test 
    if(random.random()<TRAIN_TEST):
        train=1
    else:
        train=0    
    #return
    return (key,train,split)

In [432]:
initializeAccumulators()
dataset=sc.textFile("dataset/spam.data").map(preprocessing).sortBy(lambda x: x[0])
n_row=dataset.count();

In [433]:
def calcAvg():
    i=0
    while(i<n_columns):
        averages[i]=col_sums[i].value/n_row
        i+=1
#Average
calcAvg()

In [434]:
def calcResiduals(row):
    i=0
    while(i<n_columns):       
        sigmas[i].add(math.pow(row[2][i]-averages[i],2))
        i+=1
    return row
def calcSigmas():
    i=0
    while(i<n_columns):
        sigmas[i]=math.sqrt(sigmas[i].value/float(n_row-1))
        i+=1
def normalize(row):
    i=0
    while(i<n_columns):
        row[2][i]=(row[2][i]-averages[i])/sigmas[i]
        i+=1
    return row
        
        
dataset.map(calcResiduals).collect()
calcSigmas()

In [435]:
#we normalize!
dataset=dataset.map(normalize)

In [436]:
train=dataset.filter(lambda x: x[1]==1)
test=dataset.filter(lambda x: x[1]==0)
print("Split train/test done. Train contains "+str(train.count())+" elements, Test contains "+str(test.count())+" elements")

Split train/test done. Train contains 3689 elements, Test contains 912 elements


In [3]:
dataset_RDD=sc.parallelize(dataset)
X=sc.parallelize(dataset.drop("f57", axis=1))
y=sc.parallelize(dataset["f57"])


In [4]:


X=preprocessing.normalize(dataset.drop("f57", axis=1), norm='l2')
y=dataset["f57"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)